# Functional connectomics example

## Connect to the database server

In [3]:
 # DatatJoint configuration and connection

import datajoint as dj


## NDA: schema with functional data

In [4]:
nda = dj.create_virtual_module('nda', 'microns_nda2')

Please enter DataJoint username: celiib
Please enter DataJoint password: ········
Connecting celiib@localhost:3306


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)")

## TA3: schema with structural data

In [ ]:
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')

In [ ]:
ta3 = dj.create_virtual_module('ta3', 'microns_ta3')

## Combined structural and functional pipeline

In [ ]:
dj.ERD(ta3) + dj.ERD(nda)

## Functional scans

In [ ]:
nda.Scan()

## Segmented cells in the EM volume

In [ ]:
nda.Trace * ta3.Soma

## Calcium traces 

In [ ]:
nda.Trace()

## Calcium traces for segmented cells

In [ ]:
nda.Trace & ta3.Soma

## Choose an example cell and plot receptive field

In [ ]:
# import matplotlib for plotting
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import rcParams

In [ ]:
key = {'em_id': 396}
plt.imshow((nda.MeanRF & key).fetch1('mean_rf'), cmap='gray')
plt.axis('equal')
plt.axis('off');

## Show 3D mesh for the cell

In [ ]:
# select relevant mesh fragments
fragments = ta3.Mesh.Fragment & (ta3.Soma & key)

In [ ]:
# interactive plot
import ipyvolume.pylab as p3   # for configuration, see https://github.com/maartenbreddels/ipyvolume
p3.figure(width=1024, height=1024)
for fragment in fragments:
    p3.plot_trisurf(*fragment['vertices'].T/1000, triangles=fragment['triangles'])
p3.squarelim()
p3.show()

In [ ]:
p3.screenshot()

## Find cells that synapse onto the given cell

In [ ]:
segments = ta3.Segment & (ta3.Soma & key)
preCells = ta3.Soma.proj(presyn='segment_id') & ta3.Synapse * segments.proj(postsyn='segment_id')

In [ ]:
preCells

## Presynaptic cells recorded in the same scan as the postsynaptic cell

In [ ]:
pre_cell_mask = nda.Mask & preCells & (nda.Mask & key).proj(post_id='em_id')
pre_cell_mask

## Plot Ca$^{2+}$ traces for two connected cells

In [ ]:
slice_key = (nda.Slice & pre_cell_mask).fetch('KEY')[0]
pre_trace = nda.Trace & pre_cell_mask
post_trace = (nda.Trace * nda.ScanInfo & key & slice_key).proj('trace', 'fps')

In [ ]:
pre_trace

In [ ]:
post_trace

In [ ]:
pre_trace

In [ ]:
import numpy as np
from scipy.signal import savgol_filter
from functools import partial
rcParams['figure.figsize'] = (14, 12)

fig, (a1, a2) = plt.subplots(2, sharex=True)

smooth = partial(savgol_filter, window_length=21, polyorder=7)
trace, fps = post_trace.fetch1('trace', 'fps', squeeze=True)
time = np.r_[:trace.size]/fps

legend = []
for pre in pre_trace:
    a1.plot(time, smooth(pre['trace'][0,:]) + len(legend)*2000, alpha=0.5)
    legend.append('presynaptic')
    
a1.plot(time, smooth(trace/4)-2000, 'k', alpha=0.7)
a1.set_xlim([1160, 1440])
a1.set_ylabel('fluorescence')
a1.set_yticks([])
a1.legend(legend + ['postsynaptic'])
a1.set_title('calcium signals')

pupil, speed, orientation = (nda.Treadmill * nda.Pupil * nda.Stimulus & slice_key).fetch1(
    'pupil_r', 'treadmill_speed', 'conditions', squeeze=True)
a2.plot(time, pupil/5)
a2.plot(time, abs(speed))
a2.plot(time, orientation/100 - 10)
a2.set_xlabel('time (s)')
a2.legend(('Pupil radius', 'locomotion speed', 'stimulus orientation'))
a2.set_yticks([]);